In [ ]:
# import pandas as pd
import pdfplumber
import re


In [ ]:
# Function to extract strands and sub-strands with page numbers in smaller segments
def extract_strands_sub_strands_with_page_segmented(pdf_path, start_page, end_page):
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(start_page, min(end_page, len(pdf.pages))):
            try:
                page = pdf.pages[page_number]
                tables = page.extract_tables()
                for table in tables:
                    if "Strand" in table[0] and "Sub Strand" in table[0]:
                        for row in table[1:]:
                            if row[0] and row[1]:
                                extracted_data.append({
                                    "page_number": page_number + 1,
                                    "strand": row[0],
                                    "sub_strand": row[1]
                                })
            except Exception as e:
                print(f"Error processing page {page_number + 1}: {e}")

    return extracted_data

# Example usage: Process the PDF in segments
pdf_path = '../data/raw/GRADE 6 CURRICULUM DESIGNS- MATHEMATICS.pdf'
# pdf_path = '../data/raw/GRADE 6 CURRICULUM DESIGNS- HOME SCIENCE.pdf'
total_pages = 100  # Assuming the total number of pages in the PDF
segment_size = 10  # Number of pages to process at a time

all_strands_sub_strands = []
for start_page in range(0, total_pages, segment_size):
    extracted_data = extract_strands_sub_strands_with_page_segmented(pdf_path, start_page, start_page + segment_size)
    all_strands_sub_strands.extend(extracted_data)




In [ ]:
print(all_strands_sub_strands)

In [ ]:
## Page extraction with rubric page [NOT WORKING]
def extract_strands_sub_strands_with_page_segmented(pdf_path, start_page, end_page):
    extracted_data = []
    rubric_headers = ["Indicators", "Exceeds Expectations", "Meets Expectations", "Approaches Expectations", "Below Expectations"]
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(start_page, min(end_page, len(pdf.pages))):
            try:
                page = pdf.pages[page_number]
                tables = page.extract_tables()
                for table in tables:
                    # Check if the table is for Strand and Sub Strand
                    if "Strand" in ' '.join(table[0]) and "Sub Strand" in ' '.join(table[0]):
                        for row in table[1:]:
                            if row[0] and row[1]:
                                strand_data = {
                                    "page_number": page_number + 1,
                                    "strand": row[0],
                                    "sub_strand": row[1],
                                    "rubric_start_page": None  # Default value
                                }
                                extracted_data.append(strand_data)

                    # Check each cell in the first row for rubric headers
                    for cell in table[0]:
                        if any(header in cell for header in rubric_headers):
                            ## print (cell)
                            print(cell)
                            if extracted_data:
                                extracted_data[-1]["rubric_start_page"] = page_number + 1
                                break
            except Exception as e:
                print(f"Error processing page {page_number + 1}: {e}")

    return extracted_data

# Example usage: Process the PDF in segments
pdf_path = '../data/raw/GRADE 6 CURRICULUM DESIGNS- MATHEMATICS.pdf'
total_pages = 100  # Assuming the total number of pages in the PDF
segment_size = 10  # Number of pages to process at a time

all_strands_sub_strands = []
for start_page in range(0, total_pages, segment_size):
    extracted_data = extract_strands_sub_strands_with_page_segmented(pdf_path, start_page, start_page + segment_size)
    all_strands_sub_strands.extend(extracted_data)

# Displaying a portion of the extracted data for review
print(all_strands_sub_strands)  # Displaying the first 5 entries for brevity


In [ ]:
import pdfplumber
rubric_headers = ["Indicators", "Exceeds Expectations", "Meets Expectations", "Approaches Expectations", "Below Expectations"]

def find_rubric_start_pages(pdf_path, all_strands_sub_strands, rubric_headers):
    with pdfplumber.open(pdf_path) as pdf:
        for strand_info in all_strands_sub_strands:
            start_page = strand_info.get("page_number", 0)  # Default to 0 if not found
            found_rubric = False

            for page_number in range(start_page, len(pdf.pages)):
                page = pdf.pages[page_number]
                tables = page.extract_tables()

                for table in tables:
                    first_row_text = ' '.join(cell for cell in table[0] if cell)
                    if any(header in first_row_text for header in rubric_headers):
                        strand_info["rubric_start_page"] = page_number + 1  # Page numbers are 1-indexed
                        found_rubric = True
                        break  # Found the rubric, no need to check further

                if found_rubric:
                    break  # Move to the next strand_info

            if not found_rubric:
                strand_info["rubric_start_page"] = None  # Set to None if no rubric page is found

    return all_strands_sub_strands



# Run the function with the provided data
updated_strands_sub_strands = find_rubric_start_pages(pdf_path, all_strands_sub_strands, rubric_headers)

# Displaying a portion of the updated data
print(updated_strands_sub_strands)  # Displaying the first 5 entries for brevity


In [ ]:
## WORKING
complete_data = None
def extract_additional_info(pdf_path, all_strands_sub_strands):
    with pdfplumber.open(pdf_path) as pdf:
        for index, strand_info in enumerate(all_strands_sub_strands):
            start_page = strand_info["page_number"]
            end_page = all_strands_sub_strands[index + 1]["page_number"] if index + 1 < len(all_strands_sub_strands) else len(pdf.pages) + 1
            outcomes, experiences, questions = [], [], []
            values, community_service, pcis, links_to_subjects, additional_community_service = [], [], [], [], []
            skip_remaining_tables = False  # Flag to skip remaining tables once "Indicators" is found

            for page_number in range(start_page, end_page):
                if skip_remaining_tables:
                    break  # Move to the next strand_info if "Indicators" was found in previous tables

                page = pdf.pages[page_number - 1]
                tables = page.extract_tables()

                for table in tables:
                    for row in table:
                        row_text = ' '.join(filter(None, row))  # Combine all non-empty cells in the row

                        if "Indicators" in row_text:
                            skip_remaining_tables = True
                            break  # Skip to the next strand_info
                        for cell in row:
                            if cell:
                                if "Values:" in cell:
                                    values.append(cell.split("Values:")[1].strip())
                                elif "Suggested Community Service Learning Activities:" in cell:
                                    community_service.append(cell.split("Suggested Community Service Learning Activities:")[1].strip())
                                elif "PCIs:" in cell:
                                    pcis.append(cell.split("PCIs:")[1].strip())
                                elif "Links to other subjects:" in cell:
                                    links_to_subjects.append(cell.split("Links to other subjects:")[1].strip())
                                elif "Suggested Community Service Learning Activities:" in cell:
                                    additional_community_service.append(cell.split("Suggested Community Service Learning Activities:")[1].strip())

                        # Extract other standard information if the row has 5 columns
                        if len(row) >= 5:
                            outcomes.append(row[2].replace('\n', ' ').strip() if row[2] else '')
                            experiences.append(row[3].replace('\n', ' ').strip() if row[3] else '')
                            questions.append(row[4].replace('\n', ' ').strip() if row[4] else '')

            # Add the collected data to the strand_info
            strand_info.update({
                "specific_learning_outcomes": outcomes,
                "suggested_learning_experiences": experiences,
                "key_inquiry_questions": questions,
                "values": values,
                "community_service_activities": community_service,
                "pcis": pcis,
                "links_to_other_subjects": links_to_subjects,
                "additional_community_service_activities": additional_community_service
            })

    return all_strands_sub_strands

# Extract and add additional information to the strands and sub_strands data
complete_data = extract_additional_info(pdf_path, updated_strands_sub_strands)


In [ ]:
def extract_rubric_data_complete(pdf_path, complete_data, rubric_headers):
    with pdfplumber.open(pdf_path) as pdf:
        for index, strand_info in enumerate(complete_data):
            rubric_start_page = strand_info.get("rubric_start_page")
            next_strand_start_page = complete_data[index + 1]["page_number"] if index + 1 < len(complete_data) else len(pdf.pages) + 1
            
            if rubric_start_page:
                rubrics = []
                rubric_continues = False
                
                # Loop through pages starting from rubric_start_page up to the page before next_strand_start_page
                for page_number in range(rubric_start_page, next_strand_start_page):
                    page = pdf.pages[page_number - 1]
                    tables = page.extract_tables()

                    # Process each table on the current page
                    for table in tables:
                        # If it's the first page of the rubrics or if rubric_continues is True
                        if rubric_continues or any(header in table[0] for header in rubric_headers):
                            # Set rubric_continues to False initially
                            rubric_continues = False
                            # Process each row in the table as a rubric entry
                            for row in table[1:] if any(header in table[0] for header in rubric_headers) else table:
                                # Construct the rubric entry if the row has 5 columns
                                if len(row) >= 5:
                                    rubric_entry = {
                                        "indicator_name": row[0].replace('\n', ' ').strip(),
                                        "rubrics": [
                                            {"level": "Exceeds Expectations", "statement": row[1].replace('\n', ' ').strip()},
                                            {"level": "Meets Expectations", "statement": row[2].replace('\n', ' ').strip()},
                                            {"level": "Approaches Expectations", "statement": row[3].replace('\n', ' ').strip()},
                                            {"level": "Below Expectations", "statement": row[4].replace('\n', ' ').strip()}
                                        ]
                                    }
                                    rubrics.append(rubric_entry)
                                
                            # If this table was a rubric table, set rubric_continues to True for the next page
                            if any(header in table[0] for header in rubric_headers):
                                rubric_continues = True

                # Update the strand_info with the accumulated rubric entries
                strand_info["assessment_rubrics"] = rubrics

    return complete_data

# Assuming complete_data is already defined and includes rubric start pages
# pdf_path = '/mnt/data/GRADE 6 CURRICULUM DESIGNS- MATHEMATICS.pdf'
rubric_headers = ["Indicators", "Exceeds Expectations", "Meets Expectations", "Approaches Expectations", "Below Expectations"]

# Extract rubric data
complete_data_with_rubrics = extract_rubric_data_complete(pdf_path, complete_data, rubric_headers)

# Check the last indicator name for the first rubric entry to verify correct extraction
if complete_data_with_rubrics and complete_data_with_rubrics[0]["assessment_rubrics"]:
    last_indicator = complete_data_with_rubrics[0]["assessment_rubrics"][-1]["indicator_name"]
else:
    last_indicator = "No rubric entries found"

last_indicator


In [ ]:
print(complete_data_with_rubrics)

In [ ]:
import json

# Define the path for the processed JSON file
original_file_name = "GRADE 6 CURRICULUM DESIGNS- MATHEMATICS"
processed_pdf_path = f'../data/processed/{original_file_name}.json'

# Writing the extracted data to a JSON file
with open(processed_pdf_path, 'w', encoding='utf-8') as file:
    json.dump(complete_data_with_rubrics, file, ensure_ascii=False, indent=4)




In [ ]:
# pdf = pdfplumber.open(pdf_path) 
# page = pdf.pages[16 - 1]
# tables = page.extract_tables()

In [ ]:
# tables